In [1]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)

itos = {i:char for i,char in enumerate(chars)}
stoi = {char:i for i, char in enumerate(chars)}
encode = lambda x: [stoi[char] for char in list(x)]
decode = lambda x: ''.join([itos[num] for num in list(x)])

print(encode('poor'), decode(encode("poor")))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
[54, 53, 53, 56] poor


In [2]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [3]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [4]:
block_size = 8

In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(context.tolist(), target.item())

[18] 47
[18, 47] 56
[18, 47, 56] 57
[18, 47, 56, 57] 58
[18, 47, 56, 57, 58] 1
[18, 47, 56, 57, 58, 1] 15
[18, 47, 56, 57, 58, 1, 15] 47
[18, 47, 56, 57, 58, 1, 15, 47] 58


In [6]:
torch.manual_seed(163173)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:block_size+i]for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    return x, y 

xb, yb = get_batch("train")

print("--------")
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"context: {context.tolist()}: target {target}")


--------
context: [41]: target 53
context: [41, 53]: target 52
context: [41, 53, 52]: target 57
context: [41, 53, 52, 57]: target 59
context: [41, 53, 52, 57, 59]: target 51
context: [41, 53, 52, 57, 59, 51]: target 43
context: [41, 53, 52, 57, 59, 51, 43]: target 42
context: [41, 53, 52, 57, 59, 51, 43, 42]: target 1
context: [44]: target 1
context: [44, 1]: target 48
context: [44, 1, 48]: target 53
context: [44, 1, 48, 53]: target 63
context: [44, 1, 48, 53, 63]: target 0
context: [44, 1, 48, 53, 63, 0]: target 32
context: [44, 1, 48, 53, 63, 0, 32]: target 46
context: [44, 1, 48, 53, 63, 0, 32, 46]: target 39
context: [47]: target 43
context: [47, 43]: target 44
context: [47, 43, 44]: target 1
context: [47, 43, 44, 1]: target 46
context: [47, 43, 44, 1, 46]: target 39
context: [47, 43, 44, 1, 46, 39]: target 58
context: [47, 43, 44, 1, 46, 39, 58]: target 46
context: [47, 43, 44, 1, 46, 39, 58, 46]: target 1
context: [42]: target 1
context: [42, 1]: target 40
context: [42, 1, 40]: t

In [7]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(163173)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
            
        return idx
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

        

torch.Size([32, 65]) tensor(4.4245, grad_fn=<NllLossBackward0>)


In [8]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


hGr!Zl:ddi;V,RQKnz-
FddddUyl;vqmmHICA,,awq3!;wBgv'Nsm--.q.dWnqVg;KMcTds-GMSkoG$d-Kq:s;Ku'ROg!ckKArEi?yK?NQrDwADcjk3Op$tehMCN-aJBcxJuMnp&G
g!HeEkeniaaaprkofsiZvkvkJ
g!;fa.?ebuIZykLDlGg!?MbuoawbfGGMWyb wd aXxoT;UCSbxpKYEWF
Jv,e,cAPLZDj??JcmowB LZrKAQTbB-.N.qoTVw&oSa S!bApJGmA.&lClAY&$WDmV,J.qzF-gYVN.-


In [9]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [10]:
batch_size = 32
for steps in range(10000):
    
    xb, yb = get_batch("train")
    
    logits, loss = m(xb, yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.4307258129119873


# -----------------------------------------------------------------------------------

In [15]:
B ,T ,C = 4 ,8 ,2
x = torch.randn((B, T, C))

In [16]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b ,t] = torch.mean(xprev, 0)